In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sklearn.metrics
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

data = pd.read_parquet("/mnt/DP_disk1/ht/datasets/autofe/recsys2023/recsys2023_train.parquet")
target_label = "is_installed"
data

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
55882,2522321,45,3346,22294,6767,21545,1159,27941,19203,6675,21574,27833,4473,18614,27291,2038,18162,12554,21865,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.308513,0.308513,0.308513,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.539897,0.000000,0.0,0.0,0.0,0,0
23896,2377898,45,20095,563,31686,15908,590,27941,19203,6675,19343,27833,30670,11359,20899,25365,14709,3249,20452,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,1,0,1,1,1,0,0,0,26.570643,0.000060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.398702,0.000000,0.000000,0.192820,0.000000,0.000000,2.159588,1.605064,2.994627,0.0,0.591404,0.000000,1.065663,90779409,1677.072375,0.000050,8.146457,1.058418,1.712723,0.000000,0.0,0.571121,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
10971,3173460,45,5156,22294,18971,25604,30192,27941,19203,6675,19343,32266,4473,17705,26082,29887,14709,11918,12020,29982,27961,4740,0,0,0,0,0,0,0,0,0,0,3,0,1,1,1,1,1,0,0,0,15.657012,1.620103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.909948,0.000000,0.000000,0.038564,0.038564,0.038564,2.198152,0.830089,0.302038,0.0,1.626360,0.000000,0.000000,13087224,23.439561,1.495782,7.993264,0.000000,0.000000,1.148976,0.0,2.855607,0.571121,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
85282,1930117,45,3346,22294,6767,21545,10208,27941,21621,6675,19343,32266,4473,18614,20496,11481,14709,6395,21865,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,0,0,1.272614,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.192820,0.192820,0.0,0.0,0.0,0,0
66600,2318339,45,3346,22294,5579,15908,11774,27941,21218,6675,22970,27957,4473,20634,20899,908,14709,13234,12020,29982,27961,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,1,1,0,0,0,21.673006,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.115692,0.038564,0.115692,0.347077,0.154256,0.694153,0.000000,0.000000,0.0,1.478509,0.258599,1.420884,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,3012038,66,26325,7152,21563,21545,28565,27941,21621,6675,22970,11004,4255,4230,26485,1590,25546,11918,12020,16851,30153,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,1,0,0,0,29.308692,2.225864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.762581,0.000000,0.000000,0.000000,0.347077,0.000000,0.347077,1.229138,1.756662,0.0,4.287676,0.129300,0.000000,51532956,556.191364,2.157136,8.087705,4.223591,11.651090,2.177379,0.0,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
1188,3012328,66,20095,563,31686,19475,590,27941,19203,6675,21574,11810,11402,18614,27291,14827,25546,28694,21865,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,1,1,0,0,0,28.537411,0.000801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.270802,0.038564,0.000000,0.038564,4.511996,0.000000,5.591790,1.707634,1.979011,0.0,0.147851,0.000000,0.000000,255183433,2255.701154,0.000692,8.190964,0.199623,0.488804,0.000204,0.0,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
46555,862173,66,20095,563,22861,25604,21280,27941,21218,6675,19343,11407

In [2]:
import featuretools as ft
from pyrecdp.core.utils import Timer
print(f"featuretools version is {ft.__version__}")

features_train = [i for i in data.columns if i != target_label]
day_i_y = 66
train = data[data['f_1'] < day_i_y]
test = data[data['f_1'] == day_i_y]

from featuretools.primitives import TransformPrimitive

from IPython.display import display

def produce_featuretools_entityset(es, df):
    es.add_dataframe(dataframe_name="main",
                     dataframe=df,
                     index="f_0",
                     time_index='f_1')
    return es
def modelling_features(df):
    es = ft.EntitySet("recsys2023")
    with Timer("produce entityset"):
        es = produce_featuretools_entityset(es, df)

    df, features = ft.dfs(
                    entityset=es,
                    target_dataframe_name="main",
                    verbose=True,
                    max_depth=3,)  
    return df, es, features
with Timer("enrich feature for train"):
    print(f"\n*** before enrich ***")
    print(train.dtypes)
    
    transformed_train_df, es, features = modelling_features(train)
    
    print(f"\n*** after enrich ***")
    print(transformed_train_df.dtypes)
     
print("Before Training")
display(transformed_train_df)

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/
featuretools version is 1.26.0
enrich feature for train started ...

*** before enrich ***
f_0               int64
f_1               int64
f_2               int64
f_3               int64
f_4               int64
f_5               int64
f_6               int64
f_7               int64
f_8               int64
f_9               int64
f_10              int64
f_11              int64
f_12              int64
f_13              int64
f_14              int64
f_15              int64
f_16              int64
f_17              int64
f_18              int64
f_19              int64
f_20              int64
f_21              int64
f_22              int64
f_23              int64
f_24              int64
f_25              int64
f_26              int64
f_27              int64
f_28              int64
f_29              int64
f_30              int64
f_31              int64
f_32              int64
f_33              int64
f_34           

/root/anaconda3/envs/autofe/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


Elapsed: 00:48 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

*** after enrich ***
f_1               int64
f_2               int64
f_3               int64
f_4               int64
f_5               int64
f_6               int64
f_7               int64
f_8               int64
f_9               int64
f_10              int64
f_11              int64
f_12              int64
f_13              int64
f_14              int64
f_15              int64
f_16              int64
f_17              int64
f_18              int64
f_19              int64
f_20              int64
f_21              int64
f_22              int64
f_23              int64
f_24              int64
f_25              int64
f_26              int64
f_27              int64
f_28              int64
f_29              int64
f_30              int64
f_31              int64
f_32              int6

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
f_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,45,5156,22294,18971,25604,30192,27941,21621,6675,19343,27957,4473,4230,20496,9977,18162,11918,12020,29982,27961,4740,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,46.469702,1.620103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.909948,0.077128,0.077128,0.077128,0.809845,0.809845,0.809845,0.830089,0.302038,0.0,0.000000,0.000000,0.355221,13087224,23.439561,1.495782,7.993264,0.000000,0.000000,1.148976,0.0,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
32,45,20095,563,31686,21545,590,27941,19606,6675,19343,27833,30670,4230,20496,24017,18162,3249,12020,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,1,1,0,0,0,44.001602,0.000060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.398702,0.000000,0.000000,0.000000,0.000000,0.000000,2.583793,1.605064,2.994627,0.0,3.991974,9.568178,1.065663,90779409,1677.072375,0.000050,8.146457,1.058418,1.712723,0.000000,0.0,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,1
35,45,32683,22294,19583,21545,10506,27941,21621,6675,19343,30251,8659,4230,26485,17397,18162,24285,12020,27564,29364,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0.925538,1.746335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025923,0.154256,0.154256,0.154256,0.231384,0.231384,0.578461,5.429959,0.000000,0.0,0.000000,0.000000,0.000000,757187,804.547631,1.804654,7.390880,0.000000,0.000000,2.630899,0.0,0.571121,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,1
43,45,20095,563,31686,21545,590,27941,21218,6675,22970,28360,4473,20634,20899,908,14709,8820,12020,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,1,0,1,1,1,0,0,0,5.360406,0.000060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.398702,0.192820,0.154256,0.192820,0.424205,0.077128,0.501333,1.605064,2.994627,0.0,0.147851,0.129300,0.000000,90779409,1677.072375,0.000050,8.146457,1.058418,1.712723,0.000000,0.0,1.713364,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
55,45,27426,22294,11338,21545,4104,27941,21218,6675,22970,11004,17133,4230,20899,12798,14709,29718,5774,29982,4740,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,1,0,0,0,35.247559,1.003624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.392721,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.939403,0.293381,0.0,0.739254,0.129300,0.000000,87224951,1571.595977,1.006366,7.692349,1.705995,28.178592,0.787752,0.0,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485721,65,11077,7152,18575,25604,16376,27941,19203,31372,19343,11407,4473,24696,20496,2433,18695,22244,19538,21743,15610,9968,27707,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,18.934958,0.717865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.913508,0.077128,0.038564,0.308513,0.269948,0.000000,1.889639,1.168422,0.233570,0.0,0.443553,3.103193,0.000000,52359283,2874.938335,0.775570,8.159145,0.000000,0.000000,0.735807,0.0,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
3485727,65,30256,22294,24110,25604,26145,27941,21218,31372,22970,32266,8659,24696,20496,4541,14709,24285,19937,25773,28576,28979,13631,0,0,0,0,0,0,0,0,0,3,0,1,0,1,1,1,0,0,0,18.510753,2.409223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.154953,0.154256,0.154256,0.231384,0.617025,0.578461,0.964102,0.462497,1.211954,0.0,0.147851,0.000000,1.776105,75753031,2570.634440,2.465113,7.398586,0.000000,0.000000,2.627060,0.0,3.426729,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,1,0
3485754,65,26325

In [3]:
with Timer("enrich feature for test"):
    print(f"\n*** before enrich ***")
    print(test.dtypes)
    
    transformed_test_df, es, features = modelling_features(test)
    
    print(f"\n*** after enrich ***")
    print(transformed_test_df.dtypes)
     
print("Before Training")
display(transformed_test_df)

enrich feature for test started ...

*** before enrich ***
f_0               int64
f_1               int64
f_2               int64
f_3               int64
f_4               int64
f_5               int64
f_6               int64
f_7               int64
f_8               int64
f_9               int64
f_10              int64
f_11              int64
f_12              int64
f_13              int64
f_14              int64
f_15              int64
f_16              int64
f_17              int64
f_18              int64
f_19              int64
f_20              int64
f_21              int64
f_22              int64
f_23              int64
f_24              int64
f_25              int64
f_26              int64
f_27              int64
f_28              int64
f_29              int64
f_30              int64
f_31              int64
f_32              int64
f_33              int64
f_34              int64
f_35              int64
f_36              int64
f_37              int64
f_38              int64
f_39 

/root/anaconda3/envs/autofe/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


  produce entityset took 1.3639126531779766 sec
Built 81 features
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

*** after enrich ***
f_1               int64
f_2               int64
f_3               int64
f_4               int64
f_5               int64
f_6               int64
f_7               int64
f_8               int64
f_9               int64
f_10              int64
f_11              int64
f_12              int64
f_13              int64
f_14              int64
f_15              int64
f_16              int64
f_17              int64
f_18              int64
f_19              int64
f_20              int64
f_21              int64
f_22              int64
f_23              int64
f_24              int64
f_25              int64
f_26              int64
f_27              int64
f_28              int64
f_29              int64
f_30

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32,f_33,f_34,f_35,f_36,f_37,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,f_52,f_53,f_54,f_55,f_56,f_57,f_58,f_59,f_60,f_61,f_62,f_63,f_64,f_65,f_66,f_67,f_68,f_69,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
f_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
39,66,26325,7779,21563,21545,328,27941,21218,6675,22970,11810,702,18614,20496,17123,22197,30304,21865,16851,30153,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,24.372491,2.225864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.762581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.229138,1.756662,0.0,1.626360,0.000000,0.000000,51532956,556.191364,2.157136,8.087705,4.223591,11.651090,2.177379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0
211,66,20095,563,31686,21545,590,27941,19203,6675,19343,12213,4473,30214,26082,17324,25546,12554,5774,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,1,1,0,0,0,23.832594,0.000801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.270802,0.000000,0.000000,0.000000,0.308513,0.154256,0.308513,1.707634,1.979011,0.0,0.000000,0.000000,8.880527,255183433,2255.701154,0.000692,8.190964,0.199623,0.488804,0.000204,0.000000,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
213,66,20095,563,22861,25604,21280,27941,19203,6675,19343,12213,17133,24696,26485,19550,18162,29718,5774,26758,4222,4625,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,1,1,0,0,0,16.004088,0.000801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.270802,0.462769,0.308513,0.617025,0.964102,0.848410,1.272614,1.707634,1.979011,0.0,1.034956,0.129300,1.065663,255183433,2255.701154,0.000692,8.190964,0.199623,0.488804,0.000204,0.000000,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
244,66,15471,22294,17660,21545,18841,27941,21218,6675,22970,11810,10488,24696,20899,854,14709,11022,32333,25370,15025,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,36.558737,1.731543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.736269,0.000000,0.000000,0.000000,0.771281,0.000000,1.503999,1.697569,2.214544,0.0,0.591404,11.378374,0.000000,41815218,385.573864,1.811731,7.841176,0.000000,0.000000,2.665035,0.000000,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
262,66,6581,22294,28466,21545,14597,27941,21218,6675,19343,27957,17133,17705,20496,23101,14709,29718,12020,25952,21553,0,0,0,0,0,0,0,0,0,0,0,2,1,1,1,1,1,1,0,0,0,11.183580,1.233773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.032684,0.154256,0.000000,0.154256,0.077128,0.000000,0.077128,1.196815,2.912760,0.0,2.217763,5.559887,1.776105,83602402,784.474935,1.487095,7.685511,0.000000,0.000000,2.929505,0.571121,0.571121,0.571121,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485584,66,5156,22294,13215,25604,14679,27941,18800,6675,19343,31863,17133,17705,26485,19819,18162,29718,5774,29982,27961,4740,0,0,0,0,0,0,0,0,0,0,3,0,1,0,1,1,1,0,0,0,25.413721,1.518272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.710834,0.308513,0.038564,0.424205,0.771281,0.385641,1.735383,0.922892,0.311394,0.0,6.061887,0.258599,4.262653,83256863,104.023006,1.409186,8.120913,0.000000,0.000000,1.123374,0.000000,1.142243,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,1,0
3485646,66,11077,7152,18575,21545,20137,27941,21218,6675,19343,27430,4473,18614,20496,1261,22197,12554,21865,21743,15610,9968,27707,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0.000000,0.808611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.370418,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.381249,0.183409,0.0,0.000000,0.000000,0.000000,91139269,3007.575624,0.874607,8.055544,0.000000,0.000000,0.750122,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [4]:
features_train = [i for i in transformed_train_df.columns if i != target_label]
train_df = transformed_train_df
test_df = transformed_test_df

import lightgbm as lgb
lgbm_parms = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss', 'auc'],
    'num_leaves': 63,
    'max_bin': 255,
    'num_trees': 10000,
    'min_data_in_leaf': 20,
    'min_sum_hessian_in_leaf': 5.0,
    'is_enable_sparse': True,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'early_stopping_rounds':1000,
}

with Timer(f"Train until day {day_i_y}, and valid on day {day_i_y}"):
    dtrain = lgb.Dataset(
        data=train_df[features_train], 
        label=train_df[target_label]
    )
    dvalid = lgb.Dataset(
        data=test_df[features_train], 
        label=test_df[target_label]
    )
    
    model = lgb.train(
        lgbm_parms,
        train_set=dtrain,
        valid_sets=[dtrain, dvalid],
        verbose_eval=1000,)

Train until day 66, and valid on day 66 started ...


/root/anaconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/root/anaconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 588239, number of negative: 2799641
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.123495 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5851
[LightGBM] [Info] Number of data points in the train set: 3387880, number of used features: 79
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.173630 -> initscore=-1.560113
[LightGBM] [Info] Start training from score -1.560113
Training until validation scores don't improve for 1000 rounds
[1000]	training's binary_logloss: 0.243847	training's auc: 0.924364	valid_1's binary_logloss: 0.307732	valid_1's auc: 0.891351
[2000]	training's binary_logloss: 0.238106	training's auc: 0.928207	valid_1's binary_logloss: 0.311745	valid_1's auc: 0.88709
Early stopping, best iteration is:
[1032]	training's binary_logloss: 0.243572	training's auc: 0.924552	valid_1's binar